In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
from tqdm.dask import TqdmCallback
import numpy as np
import xarray
import pandas as pd
import holoviews as hv
hv.extension("bokeh")
import colorcet as cc

hv.opts.defaults(
    hv.opts.Scatter(frame_height=400, frame_width=400, tools=["hover"], show_grid=True),
    hv.opts.Image(frame_height=400, frame_width=400, tools=["hover"], show_grid=True, colorbar=True),
    hv.opts.Path(frame_height=300, frame_width=400, tools=["hover"], show_grid=True, line_width=2),
    hv.opts.VectorField(frame_height=500, frame_width=500, show_grid=True, tools=[]),
)

In [ ]:
hv.archive.auto()

In [ ]:
import gnss_tid.synthetic as gts

In [ ]:
with TqdmCallback():
    data = gts.planar_model(snr_lim=[0, 3]).compute()

In [ ]:
def plot(trial, time):
    theta = data.dir.isel(trial=trial).item()
    dist = time * data.lam.isel(trial=trial).item() / (data.tau.isel(trial=trial).item())
    v = np.exp(1j * theta) * dist
    return (
        hv.Image(data.isel(time=time, trial=trial), vdims="image").opts(
            show_grid=True,
        ) *
        hv.Curve(([0, v.real], [0, v.imag]), kdims="x", vdims="y").opts(color="red")
    )

hv.HoloMap(
    {
        (trial, data.time.values[t]): plot(trial, t) for t in range(0, 50, 5) for trial in range(5)
    },
    kdims=["trial", "time"]
)
